In [33]:
import pandas
import matplotlib.pyplot as plt
from collections import Counter
import string
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [34]:
wf = pandas.read_csv("./news_sentiment_analysiscsv.csv")
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Source,Author,Title,Description,URL,Published At,Sentiment,Type
0,etfdailynews,MarketBeat News,"Gilead Sciences, Inc. (NASDAQ:GILD) Stake Decr...",Sei Investments Co. lessened its holdings in s...,https://www.etfdailynews.com/2024/07/16/gilead...,2024-07-16T13:42:43+00:00,positive,Science
1,dailybreeze,Donna Littlejohn,Private security contracted for Wilmington’s A...,Peak Security has operated in Wilmington in th...,https://www.dailybreeze.com/2024/07/16/private...,2024-07-16T19:10:09+00:00,negative,Business
2,marinij,Marin Independent Journal,Sports Briefs: Mill Valley LL earns redemption...,Starter Oscar Karlen threw four shutout inning...,https://www.marinij.com/2024/07/14/sports-brie...,2024-07-14T23:59:41+00:00,negative,Sports
3,krdo,Emily Arseneau,One in 30 million orange lobster saved from de...,"PUEBLO, Colo. (KRDO) - The Red Lobster in Pueb...",https://krdo.com/news/2024/07/16/one-in-30-mil...,2024-07-16T21:08:58+00:00,positive,General
4,leral.net,MD,Santé / AVC : Un neurologue alerte sur l'augme...,Le chef du service de neurologie de l'hôpital ...,https://www.leral.net/Sante-AVC-Un-neurologue-...,2024-07-14T21:08:00+00:00,neutral,General
...,...,...,...,...,...,...,...,...
3495,bostonherald,Associated Press,"Rally shooter had photos of Trump, Biden and o...",The man who tried to assassinate Trump had pho...,https://www.bostonherald.com/2024/07/18/rally-...,2024-07-18T17:33:07+00:00,negative,General
3496,gala,NaN,"Jana Ina + Giovanni Zarrella: Die ""Ü 40 Swifti...",Jana Ina und Giovanni Zarrella sind ein wahres...,https://www.gala.de/stars/jana-ina---giovanni-...,2024-07-18T09:37:00+00:00,neutral,Entertainment
3497,tribune,Saliu Gbadamosi,Court orders bank to pay Kano car dealer N34.4...,Tribune OnlineCourt orders bank to pay Kano ca...,https://tribuneonlineng.com/court-orders-bank-...,2024-07-18T18:10:09+00:00,positive,General
3498,awfulannouncing,Sean Keeley,Jeff Pearlman responds to David Wells’ critici...,Former Major League Baseball pitcher David Wel...,https://awfulannouncing.com/mlb/jeff-pearlman-...,2024-07-12T20:00:39+00:00,positive,Sports


In [35]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Sentiment']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [36]:
print(counter.most_common())

[('positive', 2134), ('neutral', 789), ('negative', 577)]


In [37]:

for i in range(len(wf)):
    if wf['Sentiment'][i] == counter.most_common()[0][0]:
        wf['Sentiment'][i] = 0
    elif wf['Sentiment'][i] == counter.most_common()[1][0]:
        wf['Sentiment'][i] = 1
    elif wf['Sentiment'][i] == counter.most_common()[2][0]:
        wf['Sentiment'][i] = 2

/tmp/ipykernel_35398/317023626.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Sentiment'][i] = 0
/tmp/ipykernel_35398/317023626.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values

In [38]:
# for i in range(len(wf['Description'])):
#     wf['Description'][i] = remove_punctuation(wf['Description'][i])
#     wf['Description'][i] = remove_stopwords(wf['Description'][i])

for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Description'], str)):
                wf = wf.drop(i)


counter = count_unique_word(wf)


num_unique_words = len(counter)



wf_training = wf[:int(len(wf) * 0.8)]
wf_testing = wf[int(len(wf) * 0.8):]



wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing



wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training


training_sentences = wf_training.Description.to_numpy()
#training labels
training_labels = wf_training.Sentiment.to_numpy()
validation_sentences = wf_testing.Description.to_numpy()
#training labels
validation_labels = wf_testing.Sentiment.to_numpy()




tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)


word_index = tokenizer.word_index




training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)


max_length = 100
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape


/tmp/ipykernel_35398/819262288.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)
/tmp/ipykernel_35398/819262288.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


((2800, 100), (700, 100))

In [39]:
#LSTM Model



model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001) #from 0.001
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [41]:
training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [42]:

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(training_labels),
    y=training_labels
)
class_weights = dict(enumerate(class_weights))

In [43]:
print(class_weights)

{0: 0.5509641873278237, 1: 1.453790238836968, 2: 2.0114942528735633}


In [44]:
wf_training['Description'].iloc[3]

'PUEBLO, Colo. (KRDO) - The Red Lobster in Pueblo recently received a special crustation that didn\'t belong in a shipment of live lobsters. The general manager says that dishwasher, Jose Romero, discovered an orange lobster. Orange lobsters are 1 in 30 million. "I have been at Red Lobster for 12 years, I always unpack theThe post One in 30 million orange lobster saved from death by Pueblo Red Lobster appeared first on KRDO.'

In [45]:
model.fit(training_padded, training_labels, epochs=30, validation_data=(validation_padded, validation_labels), class_weight=class_weights, batch_size=32, verbose=2)

Epoch 1/30


KeyboardInterrupt: 